# import

In [1]:
import warnings
warnings.filterwarnings('ignore')
import os
os.environ['TOKENIZERS_PARALLELISM'] = 'true'
from transformers import AutoModel, CLIPProcessor
from models import WhereIsFeatures
from dataset import FolderData, COCOCaptionData
import tensorflow as tf
from torch.utils.data import DataLoader
from timm.scheduler.cosine_lr import CosineLRScheduler
from torch import nn
import torch
from matplotlib import pyplot as plt
from torchvision import transforms
from PIL import Image
import numpy as np
from torchvision import datasets
from torch.nn import functional as F

2023-05-24 20:33:46.897895: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-24 20:33:47.185779: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# fns

In [2]:
def accuracy(text_embeds, image_embeds, labels):
    logits_per_image = torch.matmul(text_embeds, image_embeds.t()).t()
    probs = logits_per_image.softmax(dim=1)
    return (probs.argmax(1) == labels).float().mean()

def clip_tanh_accuracy(text_embeds, image_embeds, labels):
    logits_per_image = torch.matmul(text_embeds*2-1, (image_embeds*2-1).t()).t()
    probs = logits_per_image.softmax(dim=1)
    return (probs.argmax(1) == labels).float().mean()

# params

In [3]:
device = 'cuda'
n_epochs = 5
warmup = 4
num_workers = 4
batch_size = 16

# data

In [4]:
train_src = '/home/palm/data/coco/annotations/annotations/captions_train2017.json'
test_src = '/home/palm/data/dogs-vs-cats/train'

In [5]:
train_dataset = COCOCaptionData(train_src)
val_dataset = FolderData(test_src, size=224, mul=1)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)

In [6]:
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

In [7]:
test_texts = []
for folder in sorted(os.listdir(test_src)):
    test_texts.append(f'a photo of a {folder}')
test_inputs = processor(text=test_texts, return_tensors="pt", padding=True)
test_texts

['a photo of a cat', 'a photo of a dog']

# modules

In [42]:
class CrossMSE(nn.Module):
    def forward(self, predict, target):
        predict = predict[:, 0]
        target = target[:, 0]
        p = torch.matmul(predict, predict.t()).t().softmax(dim=1)
        q = torch.matmul(target, target.t()).t().softmax(dim=1)
        return -(p * torch.log(q + 1e-8)).sum()

In [43]:
mse = nn.MSELoss()
crossbatch = CrossMSE()
sigmoid = nn.Sigmoid()

In [55]:
clip = AutoModel.from_pretrained('openai/clip-vit-base-patch32').to(device)
for param in clip.parameters():
    param.requires_grad = False
vision_model = clip.vision_model
visual_projection = clip.visual_projection
text_projection = clip.text_projection
test_prompts = clip.text_model(**test_inputs.to(device))
test_prompts = sigmoid(text_projection(test_prompts[1]))
model = WhereIsFeatures(1500)
model = model.to(device)

# autoencoder: encoder/decoder

In [56]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
schedule = CosineLRScheduler(optimizer,
                                t_initial=5,
                                t_mul=1,
                                lr_min=5e-5,
                                decay_rate=0.1,
                                cycle_limit=1,
                                t_in_epochs=False,
                                noise_range_t=None,
                                )
for epoch in range(2):
    print('Epoch:', epoch + 1)
    model.train()
    progbar = tf.keras.utils.Progbar(3000)
    for idx, (captions, cls) in enumerate(train_loader):
        with torch.no_grad():
            captions = processor(text=captions, return_tensors="pt", padding=True)
            features = clip.text_model(**captions.to(device))
            features = text_projection(features[1])
            features = sigmoid(features)

        x = model.encode(features)
        recon = model.decode(x)
        loss = mse(recon, features)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        printlog = [('loss', loss.cpu().detach().numpy()),
                    ]
        progbar.update(idx + 1, printlog)
        if idx > 2998:
            break
    model.eval()
    progbar = tf.keras.utils.Progbar(len(test_loader))
    with torch.no_grad():
        for idx, (image, _, cls) in enumerate(test_loader):
            image = image.to(device)
            cls = cls.to(device)
            features = vision_model(image)['pooler_output']
            features = visual_projection(features)
            features = sigmoid(features)
            std_acc = accuracy(test_prompts, features, cls)
            x = model.encode(features)
            recon = model.decode(x)
            recon_acc = accuracy(test_prompts, recon, cls)
            loss = mse(recon, features)
            printlog = [('loss', loss.cpu().detach().numpy()),
                        ('std_acc', std_acc.cpu().detach().numpy()),
                        ('recon_acc', recon_acc.cpu().detach().numpy()),
                        ]
            progbar.update(idx + 1, printlog)


Epoch: 1
1250/1250 [==============================] - 43s 34ms/step - loss: 0.0021 - std_acc: 0.9907 - recon_acc: 0.9904
Epoch: 2
1250/1250 [==============================] - 43s 35ms/step - loss: 0.0016 - std_acc: 0.9913 - recon_acc: 0.9913


# autoencoder: buffer nowhere mse

In [57]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
schedule = CosineLRScheduler(optimizer,
                             warmup_t=1,
                             warmup_lr_init=1e-5,
                             t_initial=n_epochs,
                             t_mul=1,
                             lr_min=5e-5,
                             decay_rate=0.1,
                             cycle_limit=1,
                             t_in_epochs=False,
                             noise_range_t=None,
                                )
for epoch in range(n_epochs):
    print('Epoch:', epoch + 1)
    model.train()
    progbar = tf.keras.utils.Progbar(3000)
    for idx, (captions, cls) in enumerate(train_loader):
        with torch.no_grad():
            captions = processor(text=captions, return_tensors="pt", padding=True)
            features = clip.text_model(**captions.to('cuda'))
            features = text_projection(features[1])
            features = sigmoid(features)
            x = model.encode(features)

        x, ecd, gt = model.where(x, False)
        loss = mse(x, gt)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        printlog = [('loss', loss.cpu().detach().numpy())
                    ]
        progbar.update(idx + 1, printlog)
        if idx > 2998:
            break
    model.eval()
    progbar = tf.keras.utils.Progbar(len(test_loader))
    with torch.no_grad():
        for idx, (image, _, cls) in enumerate(test_loader):
            image = image.to(device)
            cls = cls.to(device)
            features = vision_model(image)['pooler_output']
            features = visual_projection(features)
            features = sigmoid(features)
            std_acc = accuracy(test_prompts, features, cls)
            prompts_ecd = model.encode(test_prompts)
            _, prompts_ecd, _ = model.where(prompts_ecd, False)
            x = model.encode(features)
            x, ecd, gt = model.where(x, False)
            buffer_acc = accuracy(prompts_ecd[:, 0], ecd[:, 0], cls)
            recon = model.decode(x)
            recon_acc = accuracy(test_prompts, recon, cls)
            loss = mse(x, gt)
            printlog = [('loss', loss.cpu().detach().numpy()),
                        ('std_acc', std_acc.cpu().detach().numpy()),
                        ('recon_acc', recon_acc.cpu().detach().numpy()),
                        ('buffer_acc', buffer_acc.cpu().detach().numpy()),
                        ]
            progbar.update(idx + 1, printlog)


Epoch: 1
1250/1250 [==============================] - 46s 37ms/step - loss: 5.4906 - std_acc: 0.9911 - recon_acc: 0.5663 - buffer_acc: 0.9785
Epoch: 2
1250/1250 [==============================] - 47s 37ms/step - loss: 5.4759 - std_acc: 0.9916 - recon_acc: 0.7183 - buffer_acc: 0.9891
Epoch: 3
1250/1250 [==============================] - 47s 37ms/step - loss: 5.4655 - std_acc: 0.9913 - recon_acc: 0.6589 - buffer_acc: 0.9863
Epoch: 4
1250/1250 [==============================] - 46s 37ms/step - loss: 5.4635 - std_acc: 0.9908 - recon_acc: 0.6685 - buffer_acc: 0.9886
Epoch: 5
1250/1250 [==============================] - 46s 37ms/step - loss: 5.4581 - std_acc: 0.9907 - recon_acc: 0.5412 - buffer_acc: 0.9897


# autoencoder: buffer nowhere crossbatch

In [29]:
predict = x[:, 0]
target = ecd[:, 0]
p = torch.matmul(predict, predict.t()).t().softmax(dim=1)
q = torch.matmul(target, target.t()).t().softmax(dim=1)
l = -(p * q.log()).sum()

In [31]:
l

tensor(20.4948, device='cuda:0')

In [58]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
schedule = CosineLRScheduler(optimizer,
                             warmup_t=1,
                             warmup_lr_init=1e-5,
                             t_initial=n_epochs,
                             t_mul=1,
                             lr_min=5e-5,
                             decay_rate=0.1,
                             cycle_limit=1,
                             t_in_epochs=False,
                             noise_range_t=None,
                                )
for epoch in range(n_epochs):
    print('Epoch:', epoch + 1)
    model.train()
    progbar = tf.keras.utils.Progbar(3000)
    for idx, (captions, cls) in enumerate(train_loader):
        with torch.no_grad():
            captions = processor(text=captions, return_tensors="pt", padding=True)
            features = clip.text_model(**captions.to('cuda'))
            features = text_projection(features[1])
            features = sigmoid(features)
            x = model.encode(features)

        x, ecd, gt = model.where(x, False)
        loss = crossbatch(x, ecd) + mse(x, gt)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        printlog = [('loss', loss.cpu().detach().numpy())
                    ]
        progbar.update(idx + 1, printlog)
        if idx > 2998:
            break
    model.eval()
    progbar = tf.keras.utils.Progbar(len(test_loader))
    with torch.no_grad():
        for idx, (image, _, cls) in enumerate(test_loader):
            image = image.to(device)
            cls = cls.to(device)
            features = vision_model(image)['pooler_output']
            features = visual_projection(features)
            features = sigmoid(features)
            std_acc = accuracy(test_prompts, features, cls)
            prompts_ecd = model.encode(test_prompts)
            _, prompts_ecd, _ = model.where(prompts_ecd, False)
            x = model.encode(features)
            x, ecd, gt = model.where(x, False)
            buffer_acc = accuracy(prompts_ecd[:, 0], ecd[:, 0], cls)
            recon = model.decode(x)
            recon_acc = accuracy(test_prompts, recon, cls)
            loss = mse(x, gt)
            printlog = [('loss', loss.cpu().detach().numpy()),
                        ('std_acc', std_acc.cpu().detach().numpy()),
                        ('recon_acc', recon_acc.cpu().detach().numpy()),
                        ('buffer_acc', buffer_acc.cpu().detach().numpy()),
                        ]
            progbar.update(idx + 1, printlog)


Epoch: 1
1250/1250 [==============================] - 46s 37ms/step - loss: 5.4768 - std_acc: 0.9909 - recon_acc: 0.5023 - buffer_acc: 0.9883
Epoch: 2
1250/1250 [==============================] - 46s 37ms/step - loss: 5.5082 - std_acc: 0.9914 - recon_acc: 0.5000 - buffer_acc: 0.9893
Epoch: 3
1250/1250 [==============================] - 46s 37ms/step - loss: 5.4968 - std_acc: 0.9920 - recon_acc: 0.5011 - buffer_acc: 0.9892
Epoch: 4
1250/1250 [==============================] - 46s 37ms/step - loss: 5.4907 - std_acc: 0.9917 - recon_acc: 0.5290 - buffer_acc: 0.9862
Epoch: 5
1250/1250 [==============================] - 46s 37ms/step - loss: 5.5680 - std_acc: 0.9919 - recon_acc: 0.5110 - buffer_acc: 0.9860


# autoencoder: buffer where

In [59]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
schedule = CosineLRScheduler(optimizer,
                             warmup_t=1,
                             warmup_lr_init=1e-5,
                             t_initial=n_epochs,
                             t_mul=1,
                             lr_min=5e-5,
                             decay_rate=0.1,
                             cycle_limit=1,
                             t_in_epochs=False,
                             noise_range_t=None,
                                )
for epoch in range(n_epochs):
    print('Epoch:', epoch + 1)
    model.train()
    progbar = tf.keras.utils.Progbar(3000)
    for idx, (captions, cls) in enumerate(train_loader):
        with torch.no_grad():
            captions = processor(text=captions, return_tensors="pt", padding=True)
            features = clip.text_model(**captions.to('cuda'))
            features = text_projection(features[1])
            features = sigmoid(features)
            x = model.encode(features)

        x, ecd, gt = model.where(x, True)
        loss = crossbatch(x, ecd) + mse(x, gt)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        printlog = [('loss', loss.cpu().detach().numpy()),
                    ]
        progbar.update(idx + 1, printlog)
        if idx > 2998:
            break
    model.eval()
    progbar = tf.keras.utils.Progbar(len(test_loader))
    with torch.no_grad():
        for idx, (image, _, cls) in enumerate(test_loader):
            image = image.to(device)
            cls = cls.to(device)
            features = vision_model(image)['pooler_output']
            features = visual_projection(features)
            features = sigmoid(features)
            std_acc = accuracy(test_prompts, features, cls)
            prompts_ecd = model.encode(test_prompts)
            _, prompts_ecd, _ = model.where(prompts_ecd, True)
            x = model.encode(features)
            x, ecd, gt = model.where(x, True)
            buffer_acc = accuracy(prompts_ecd[:, 0], ecd[:, 0], cls)
            buffer_acc_tanh = accuracy(prompts_ecd[:, 0]*2-1, ecd[:, 0]*2-1, cls)
            loss = mse(x, gt)
            printlog = [('loss', loss.cpu().detach().numpy()),
                        ('std_acc', std_acc.cpu().detach().numpy()),
                        ('sigmoid_acc', buffer_acc.cpu().detach().numpy()),
                        ('tanh_acc', buffer_acc_tanh.cpu().detach().numpy()),
                        ]
            progbar.update(idx + 1, printlog)


Epoch: 1
1250/1250 [==============================] - 46s 36ms/step - loss: 5.5014 - std_acc: 0.9909 - sigmoid_acc: 0.9762 - tanh_acc: 0.9803
Epoch: 2
1250/1250 [==============================] - 46s 36ms/step - loss: 5.4976 - std_acc: 0.9910 - sigmoid_acc: 0.9748 - tanh_acc: 0.9791
Epoch: 3
1250/1250 [==============================] - 46s 36ms/step - loss: 5.4981 - std_acc: 0.9911 - sigmoid_acc: 0.9746 - tanh_acc: 0.9798
Epoch: 4
1250/1250 [==============================] - 46s 36ms/step - loss: 5.5013 - std_acc: 0.9907 - sigmoid_acc: 0.9752 - tanh_acc: 0.9791
Epoch: 5
1250/1250 [==============================] - 46s 37ms/step - loss: 5.4982 - std_acc: 0.9913 - sigmoid_acc: 0.9744 - tanh_acc: 0.9786


# tanh eval

In [60]:
model.eval()
progbar = tf.keras.utils.Progbar(len(test_loader))
with torch.no_grad():
    for idx, (image, _, cls) in enumerate(test_loader):
        image = image.to(device)
        cls = cls.to(device)
        features = vision_model(image)['pooler_output']
        features = visual_projection(features)
        features = sigmoid(features)
        std_acc = accuracy(test_prompts, features, cls)
        prompts_ecd = model.encode(test_prompts)
        _, prompts_ecd, _ = model.where(prompts_ecd, True)
        x = model.encode(features)
        x, ecd, gt = model.where(x, True)
        buffer_acc = accuracy(prompts_ecd[:, 0]*2-1, ecd[:, 0]*2-1, cls)
        recon = model.decode(x)
        recon_acc = accuracy(test_prompts, recon, cls)
        loss = mse(x, gt)
        printlog = [('loss', loss.cpu().detach().numpy()),
                    ('std_acc', std_acc.cpu().detach().numpy()),
                    ('recon_acc', recon_acc.cpu().detach().numpy()),
                    ('buffer_acc', buffer_acc.cpu().detach().numpy()),
                    ]
        progbar.update(idx + 1, printlog)


1250/1250 [==============================] - 48s 38ms/step - loss: 5.5010 - std_acc: 0.9901 - recon_acc: 0.6163 - buffer_acc: 0.9795


In [48]:
text_embed = torch.rand((50, 4))
image_embed = torch.rand((50, 4))


In [57]:
labels = (torch.rand(50) * 50 + 50).int()

In [58]:
labels

tensor([52, 51, 87, 84, 78, 71, 99, 98, 60, 52, 79, 70, 87, 78, 83, 52, 63, 93,
        93, 51, 65, 63, 82, 65, 77, 98, 54, 52, 99, 94, 90, 74, 97, 71, 77, 58,
        99, 51, 67, 59, 76, 50, 80, 68, 86, 58, 77, 86, 90, 92],
       dtype=torch.int32)

In [64]:
probs = (text_embed @ text_embed.t()).softmax(1)
probs.argmax(1)

tensor([21, 21, 21, 45, 21, 21, 21,  9, 19, 21, 21, 19, 21, 21, 21, 19, 19, 21,
        21, 21, 45, 21, 21, 21, 21,  9, 21, 21, 21, 45, 21,  9, 21, 21,  9,  9,
         9, 21, 19, 21, 21, 21, 45, 21, 45, 45,  9, 45, 45, 19])

In [65]:
labels[probs.argmax(1)]

tensor([63, 63, 63, 58, 63, 63, 63, 52, 51, 63, 63, 51, 63, 63, 63, 51, 51, 63,
        63, 63, 58, 63, 63, 63, 63, 52, 63, 63, 63, 58, 63, 52, 63, 63, 52, 52,
        52, 63, 51, 63, 63, 63, 58, 63, 58, 58, 52, 58, 58, 51],
       dtype=torch.int32)

In [62]:
labels[probs.argmax(1)]==labels

tensor([False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False,  True, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False,  True, False, False, False, False])